In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/benvictoria17/Finance/main/dataset/US%20Health%20Insurance%20Dataset/insurance.csv")

In [4]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [6]:
data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [7]:
data["age"].value_counts()

18    69
19    68
50    29
51    29
47    29
46    29
45    29
20    29
48    29
52    29
22    28
49    28
54    28
53    28
21    28
26    28
24    28
25    28
28    28
27    28
23    28
43    27
29    27
30    27
41    27
42    27
44    27
31    27
40    27
32    26
33    26
56    26
34    26
55    26
57    26
37    25
59    25
58    25
36    25
38    25
35    25
39    25
61    23
60    23
63    23
62    23
64    22
Name: age, dtype: int64

In [8]:
data['sex'].value_counts()

male      676
female    662
Name: sex, dtype: int64

In [9]:
sex= pd.get_dummies(data['sex'])

In [11]:
data["smoker"].value_counts()

no     1064
yes     274
Name: smoker, dtype: int64

In [12]:
smoker = pd.get_dummies(data['smoker'])

In [13]:
corr = data.corr()

In [14]:
corr["charges"]

age         0.299008
bmi         0.198341
children    0.067998
charges     1.000000
Name: charges, dtype: float64

In [15]:
d = pd.concat([sex,smoker],axis =1)

In [16]:
df = data[['age','bmi','children']]

In [17]:
x = pd.concat([d,df],axis = 1)

In [18]:
x

,female,male,no,yes,age,bmi,children
0,1,0,0,1,19,27.900,0
1,0,1,1,0,18,33.770,1
2,0,1,1,0,28,33.000,3
3,0,1,1,0,33,22.705,0
4,0,1,1,0,32,28.880,0
...,...,...,...,...,...,...,...
1333,0,1,1,0,50,30.970,3
1334,1,0,1,0,18,31.920,0
1335,1,0,1,0,18,36.850,0
1336,1,0,1,0,21,25.800,0


In [19]:
y = data["charges"]
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

In [20]:
from xgboost import XGBRegressor
model = XGBRegressor()
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20)
model.fit(xtrain,ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [21]:
model.score(xtrain,ytrain)

0.9947327728684774

In [22]:
model.score(xtest,ytest)

0.7529617229481618

In [23]:
from sklearn.model_selection import RandomizedSearchCV
param = {'gamma' :[0.5,1,1.5,2,5],
         'min_child_weight':[1,5,6,7,10],
         'subsample':[0.3,0.4,0.5,0.8,1.0],
         'colsample_bytree':[0.6,0.7,0.8,0.9],
         'max_depth':[2,3,4,5]
    
    
    
}
randomcv =  RandomizedSearchCV(estimator = model,param_distributions= param,cv = 5,n_iter = 5,
                              scoring = 'roc_auc',n_jobs = 1,verbose = 3,return_train_score = True,
                              random_state = 121)
randomcv.fit(xtrain,ytrain)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END colsample_bytree=0.8, gamma=2, max_depth=3, min_child_weight=10, subsample=0.4;, score=(train=nan, test=nan) total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, gamma=2, max_depth=3, min_child_weight=10, subsample=0.4;, score=(train=nan, test=nan) total time=   0.1s


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceb

[CV 3/5] END colsample_bytree=0.8, gamma=2, max_depth=3, min_child_weight=10, subsample=0.4;, score=(train=nan, test=nan) total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, gamma=2, max_depth=3, min_child_weight=10, subsample=0.4;, score=(train=nan, test=nan) total time=   0.1s
[CV 5/5] END colsample_bytree=0.8, gamma=2, max_depth=3, min_child_weight=10, subsample=0.4;, score=(train=nan, test=nan) total time=   0.1s


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceb

[CV 1/5] END colsample_bytree=0.6, gamma=5, max_depth=4, min_child_weight=7, subsample=1.0;, score=(train=nan, test=nan) total time=   0.1s
[CV 2/5] END colsample_bytree=0.6, gamma=5, max_depth=4, min_child_weight=7, subsample=1.0;, score=(train=nan, test=nan) total time=   0.1s


...

/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceb

[CV 4/5] END colsample_bytree=0.9, gamma=1, max_depth=3, min_child_weight=5, subsample=0.4;, score=(train=nan, test=nan) total time=   0.1s
[CV 5/5] END colsample_bytree=0.9, gamma=1, max_depth=3, min_child_weight=5, subsample=0.4;, score=(train=nan, test=nan) total time=   0.1s
[CV 1/5] END colsample_bytree=0.9, gamma=0.5, max_depth=2, min_child_weight=10, subsample=1.0;, score=(train=nan, test=nan) total time=   0.1s


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: continuous format is not supported

  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceb

[CV 2/5] END colsample_bytree=0.9, gamma=0.5, max_depth=2, min_child_weight=10, subsample=1.0;, score=(train=nan, test=nan) total time=   0.1s
[CV 3/5] END colsample_bytree=0.9, gamma=0.5, max_depth=2, min_child_weight=10, subsample=1.0;, score=(train=nan, test=nan) total time=   0.1s
[CV 4/5] END colsample_bytree=0.9, gamma=0.5, max_depth=2, min_child_weight=10, subsample=1.0;, score=(train=nan, test=nan) total time=   0.1s
[CV 5/5] END colsample_bytree=0.9, gamma=0.5, max_depth=2, min_child_weight=10, subsample=1.0;, score=(train=nan, test=nan) total time=   0.1s


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          enable_categorical=False, gamma=0,
                                          gpu_id=-1, importance_type=None,
                                          interaction_constraints='',
                                          learning_rate=0.300000012,
                                          max_delta_step=0, max_depth=6,
                                          min_child_weight=1, missing=nan,
                                          monotone_constraints='()',
                                          n_estimato...
                                          reg_lambda=1, scale_pos_weight=1,
                                          subsample=1, tree_method='exac

In [24]:
randomcv.best_params_

{'subsample': 0.4,
 'min_child_weight': 10,
 'max_depth': 3,
 'gamma': 2,
 'colsample_bytree': 0.8}

In [25]:
model1 = XGBRegressor(subsample = 0.4,
 min_child_weight = 10,
 max_depth =  3,
  gamma = 2,
 colsample_bytree = 0.8)
model1.fit(xtrain,ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=2, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=3, min_child_weight=10, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.4,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [26]:
model1.score(xtest,ytest)

0.8068682739377654